# Project Where to play?

In [2]:
import pandas as pd
import numpy as np
import requests

### Get data of popular places from Foursquare API

In [154]:
CLIENT_ID = 'ND0AF0KVLBEM341PVLKK314M0XKG1CLGTTN42BCT2KJLGQT2' # your Foursquare ID
CLIENT_SECRET = 'FILOMWFFVSVZ2RE4DQQAUXTKT5P2RWQBEGAMYVB4NRPKUON3' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [195]:
def getVenues(city, categoryId, radius=1000, limit=50, sortByPopularity=1):

    # create the API request URL
    url = f"https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&near={city}&radius={radius}&limit={limit}&             categoryId{section}&sortByPopularity{sortByPopularity}"

    # make the GET request
    results = requests.get(url).json()['response']['groups'][0]['items']
    venues=pd.DataFrame(columns=['popularity_ranking', 'name', 'category'])

    for i, item in enumerate(results):
        venues.loc[i, 'popularity_ranking'] = i+1
        venues.loc[i, 'name'] = item['venue']['name']
        venues.loc[i, 'category'] = item['venue']['categories'][0]['name']
        venues.loc[i, 'latitudes'] = item['venue']['location']['lat']
        venues.loc[i, 'longitudes'] = item['venue']['location']['lng']
    return venues, results

In [196]:
df, results = getVenues("Amsterdam", "56aa371be4b08b9a8d5734db")

In [134]:
city = "Amsterdam"
radius=2000
limit=50
categoryId="4bf58dd8d48988d116941735" # Id of the food category
sortByPopularity=1

# create the API request URL
url = f"https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&near={city}&radius={radius}&             catetgoryId{categoryId}"

# make the GET request
results = requests.get(url).json()['response']['groups'][0]['items']
venues=pd.DataFrame(columns=['popularity_ranking', 'name', 'category'])

for i, item in enumerate(results):
    venues.loc[i, 'popularity_ranking'] = i+1
    venues.loc[i, 'name'] = item['venue']['name']
    venues.loc[i, 'category'] = item['venue']['categories'][0]['name']
venues

,popularity_ranking,name,category
0,1,De Bierkoning,Beer Store
1,2,Kaagman & Kortekaas,French Restaurant
2,3,Proeflokaal De Drie Fleschjes,Bar
3,4,W Amsterdam,Hotel
4,5,Caffè Il Momento,Café
5,6,Tales & Spirits,Cocktail Bar
6,7,Hummingbird,Coffee Shop
7,8,Melly's Espresso - Cookies Bar,Dessert Shop
8,9,Proeflokaal Arendsnest,Beer Bar
9,10,Prik,Gay Bar


### Visualize popular places

In [201]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [198]:
df.head()

,popularity_ranking,name,category,latitudes,longitudes
0,1,De Bierkoning,Beer Store,52.372404,4.889795
1,2,Proeflokaal De Drie Fleschjes,Bar,52.374203,4.892239
2,3,Kaagman & Kortekaas,French Restaurant,52.374878,4.892455
3,4,W Amsterdam,Hotel,52.372787,4.890006
4,5,Caffè Il Momento,Café,52.374682,4.889018


In [216]:
# initialize for Amsterdam
latitude = 52.38
longitude = 4.90

# create map
map_amsterdam = folium.Map(location=[latitude, longitude], zoom_start=14)

# set color scheme for the popularity rankings
colors_array = cm.rainbow(np.linspace(0, 1, 51))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, popularity_ranking, name, category in zip(df['latitudes'], df['longitudes'], df['popularity_ranking'], df['name'], df['category']):
    label = folium.Popup(str(name) + " (" + str(category) + ") , ranking:" + str(popularity_ranking), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[popularity_ranking],
        fill=True,
        fill_color=rainbow[popularity_ranking],
        fill_opacity=0.7).add_to(map_amsterdam)
       
map_amsterdam